# Artigo utilizado como base para a solução
https://research.ijcaonline.org/volume58/number17/pxc3883886.pdf

# Requisitos da atividade
1) implementar o gerador dos labirintos;

2) implementar o algoritmo genético (AG) com estruturas de dados e parâmetros necessários;

3) executar o AG num labirinto 10x10;

4) traçar um gráfico da evolução do AG (gerações versus fitness da população) usando a biblioteca matplotlib;

5) desenhar uma figura do labirinto e da melhor solução encontrada (pode ser em modo texto);

6) (Bonus) Variar os parâmetros do AG e comentar o efeito sobre a velocidade de convergência da solução.

### Grupo
* Alexandre Ribeiro
* Flávio Farias
* Henrique Arriel

# Imports e constantes

In [ ]:
import random
from random import randint, choice
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from time import sleep

SEED = 1123581321 #seed utilizada para a randomização ser igual em todas execuções do notebook. A modificação da seed pode trazer efeitos colaterais para algumas análises.
rep = { "cross": '#', 'vwall': '#', 'hwall': '#', 'step': '\033[95m+\033[00m', 'start': '\033[92mS\033[00m', 'finish': '\033[91mF\033[00m' }

# Gerador de labirintos (A-Mazer)

#### Gerar labirinto

In [ ]:
  def maze_maker(n):
    maze = [[[0,0,0,0,0] for line in range(n)] for column in range(n)]
    doors = ['LEFT', 'RIGHT', 'TOP', 'BOTTOM']
    for row in range(1,n-1):
      for col in range(1,n-1):
        for door in doors:
          opened_door = randint(0,1)
          if(opened_door):
            open_door(maze, (row, col), door)

    return maze

  def validate_cell(line,column, maze_size):
    return ((0 <= line < maze_size) and (0 <= column < maze_size))

  def start_finish_cells(maze_size):
    start = randint(0,maze_size-1)
    finish = randint(0,maze_size-1)
    return start, finish

  def open_door(maze, coordinates, door):
    x, y = coordinates
    if (door == "LEFT"):
      if(validate_cell(x, y, len(maze)) and validate_cell(x, y-1, len(maze))):
        maze[x][y][0], maze[x][y-1][1] = 1, 1
    elif (door == "RIGHT"):
      if(validate_cell(x, y, len(maze)) and validate_cell(x, y+1, len(maze))):
        maze[x][y][1], maze[x][y+1][0] = 1, 1
    elif (door == "TOP"): 
      if(validate_cell(x, y, len(maze)) and validate_cell(x-1, y, len(maze))):
        maze[x][y][2], maze[x-1][y][3] = 1, 1
    elif (door == "BOTTOM"): 
      if(validate_cell(x, y, len(maze)) and validate_cell(x+1, y, len(maze))):
        maze[x][y][3], maze[x+1][y][2] = 1, 1

#### Flood fill

In [ ]:
def flood_fill(maze, row, col, label):
  if(maze[row][col][4] == label):
    return
  else:
    maze[row][col][4] = label
    #LEFT
    if(maze[row][col][0] == 1): flood_fill(maze, row,col-1, label)
    #RIGHT
    if(maze[row][col][1] == 1): flood_fill(maze, row, col+1, label)
    #TOP
    if(maze[row][col][2] == 1): flood_fill(maze, row-1, col, label)
    #BOTTOM
    if(maze[row][col][3] == 1): flood_fill(maze, row+1, col, label)

#### Junção de paredes adjacentes

In [ ]:
#this function considers current and parent as neighbors
def came_from(current, parent):
  coord_to_text = { (0, 1): 'LEFT', (0, -1): 'RIGHT', (1, 0): 'TOP', (-1, 0): 'BOTTOM' }
  x, y = current
  px, py = parent
  from_ = (x-px, y-py)
  
  return coord_to_text[from_]

def text_to_step(text): 
  steps = {'LEFT': 0, 'RIGHT': 1, 'TOP': 2, 'BOTTOM': 3}
  return steps[text]

def step_to_text(step):
  texts = {0: 'LEFT', 1: 'RIGHT', 2: 'TOP', 3: 'BOTTOM'}
  return texts[step]

#this function doesnt consider whether door is open
def get_neighbors(maze, node):
  x, y = node
  doors = [0, 1, 2, 3]
  door_to_coord = { 0: (0, -1), 1: (0, 1), 2: (-1, 0), 3: (1, 0) }
  neighbors = []
  for door in doors:
      neighbor = (x + door_to_coord[door][0], y + door_to_coord[door][1])
      if (validate_cell(neighbor[0], neighbor[1], len(maze))):
        neighbors.append(neighbor)
  return neighbors

def merge_adjacent(maze, row, col):
  stack = []
  visited = set()
  parent = None
  stack.append(((row, col), parent))

  while(stack):
    current, parent = stack.pop()
    if (current in visited): continue
    if (maze[current[0]][current[1]][4] == 2):
      open_door(maze, current, came_from(current, parent))
      return True

    visited.add(current)
    neighbors = get_neighbors(maze, current)
    for neighbor in neighbors: 
      if (maze[neighbor[0]][neighbor[1]][4] != 0): 
        stack.append((neighbor, current))
  return False

#### Junção de paredes randômicas

In [ ]:
#get a random cell reachable from the start, inclusive
def random_cell(maze, start, except_=[]):
  stack = []
  visited = set()
  parent = None
  stack.append((start, parent))

  while(stack):
    current, parent = stack.pop()
    if (current in visited): continue
    visited.add(current)
    neighbors = get_neighbors(maze, current)
    for neighbor in neighbors: 
      if (maze[neighbor[0]][neighbor[1]][4] == 1): 
        stack.append((neighbor, current))

  [visited.remove(node) for node in except_ if node in visited]
  return choice(list(visited)) if visited else False

#get a valid random door
#cell e neighbors podem não ter uma porta entre si
def random_door(maze, cell):
  neighbors = get_neighbors(maze, cell)
  neighbor_cell = choice(neighbors)
  neighbors.remove(neighbor_cell)
  while(neighbors and maze[neighbor_cell[0]][neighbor_cell[1]][4]):
    neighbor_cell = choice(neighbors)
    neighbors.remove(neighbor_cell)
  
  return neighbor_cell if not maze[neighbor_cell[0]][neighbor_cell[1]][4] else False


def open_random_door(maze, start):
  cell = random_cell(maze, start)
  door = random_door(maze, cell)
  tried = [] #list of cells that there are not doors to open
  
  while(not door and cell):
    cell = random_cell(maze, start, tried)
    door = random_door(maze, cell)
  if (not cell): print("fully opened")
  else: 
    open_door(maze, cell, step_to_text(text_to_step(came_from(cell, door))))


#### Verifica e torna o labirinto viável

In [ ]:
def valid_maze(maze, maze_size, finish):
  return maze[finish][maze_size-1][4] == 1

def make_maze_viable(maze, start, finish):
  maze_size = len(maze)
  flood_fill(maze, start, 0, 1)

  if (not valid_maze(maze, maze_size, finish)): 
    flood_fill(maze, finish, maze_size-1, 2)

    while(not valid_maze(maze, maze_size, finish) ):

      merged = merge_adjacent(maze, start, 0)
      if (merged):
        flood_fill(maze, start, 0, 0)
        flood_fill(maze, start, 0, 1)
      else:
        open_random_door(maze, (start, 0))
        flood_fill(maze,start, 0, 0)
        flood_fill(maze, start, 0, 1)

# Algoritmo genético

#### Cromossomo

O cromossomo do algoritmo genético é representado como uma lista de tamanho inicial N*2, em que N é o tamanho do labirinto, com cada elemento da lista representando um passo no labirinto.


*   0 -> Andou para esquerda
*   1 -> Andou para direita
*   2 -> Andou para cima
*   3 -> Andou para baixo
---
Exemplo:
Dado um labirinto de tamanho 4, um cromossomo da população inicial seria: \[0,2,3,1,0,2,2,3\]



#### População inicial

A população inicial é gerada de forma randômica. Dado um tamanho de população X, são criados X cromossomos de tamanho N*2.

---
Exemplo:

X = 5

N = 2

População inicial: 

\[1,2,3,0\], \[2,3,1,0\], \[0,0,2,1\], \[3,2,3,2\], \[3,1,2,0\]

In [ ]:
def init_population(maze_size, population_size):
  chromosome_size = maze_size*2
  population = []
  for i in range(population_size):
    new_chromosome = [random.randint(0,3) for _ in range(chromosome_size)]
    population.append(new_chromosome)
  
  return population

#### Cálculo de Fitness

In [ ]:
def get_coordinates_and_distance(chromosome, maze, start):
  coordinates, moves = [start, 0], []

  for step in chromosome:
    if(maze[coordinates[0]][coordinates[1]][step]):
      coordinates[(int(step / 2) + 1) % 2] += step % 2 if step % 2 else -1
      moves.append((coordinates[0], coordinates[1]))

  return (coordinates, moves)

# Using Euclidean distance
def euclidean_fitness(chromosome, maze, start, end):
  start_coordinates = np.array((start, 0))
  end_coordinates = np.array((end, len(maze)-1))
  coordinates, _ = get_coordinates_and_distance(chromosome, maze, start)

  # Euclidean distance
  coordinates = np.array(coordinates)
  cc_sc = np.linalg.norm(coordinates - start_coordinates)
  fc_cc = np.linalg.norm(end_coordinates - coordinates)

  if(fc_cc == 0):
    return -1

  return cc_sc/fc_cc*100

# Using Manhattan Distance
def manhattan_fitness(chromosome, maze, start, end):
  start_coodinates = (start, 0)
  end_coordiantes = (end, len(maze)-1)
  coordinates, _ = get_coordinates_and_distance(chromosome, maze, start)

  cc_sc = abs(coordinates[0] - start_coodinates[0]) + abs(coordinates[1] - start_coodinates[1])
  fc_cc = abs(end_coordiantes[0] - coordinates[0]) + abs(end_coordiantes[1] - coordinates[1])

  if(fc_cc == 0):
    return -1

  return cc_sc/fc_cc*100

#### Crossover

O método de crossover recebe dois cromossomos pais e os utiliza para a geração de dois cromossomos filhos.

O primeiro filho é resultante da adição de cada passo dos pais e o segundo filho é resultante da subtração de cada passo dos pais.

É feito o módulo 4 de cada passo dos cromossomos filhos para eles se manterem no limite de 0 a 3.

---
Exemplo:

\[0,1,2\] + \[1,1,1\] = \[1,2,3\]

\[0,1,2\] - \[1,1,1\] = \[1,0,1\]

In [ ]:
def crossover(parent_1, parent_2):
  first_child = []
  second_child = []
  for i in range(len(parent_1)):
    add_step = (parent_1[i] + parent_2[i]) % 4
    sub_step = abs(parent_1[i] - parent_2[i]) % 4
    first_child.append(add_step)
    second_child.append(sub_step)
  return (first_child,second_child)

#### Mutação

O método de mutação recebe um cromossomo e faz uma modificação em um dos seus passos. É escolhido um passo aleatório e esse passo é modificado.

---
Exemplo:
\[1,2,2\] -> \[1,3,2\]

In [ ]:
def mutation(chromosome):
  new_chromosome = chromosome[:]
  random_chromosome_bit = random.randint(0, len(new_chromosome) - 1)

  values = [num for num in range(0, 4) if num != new_chromosome[random_chromosome_bit]]
  random_bit = random.choice(values)
  
  new_chromosome[random_chromosome_bit] = random_bit

  return new_chromosome

# Execução do algoritmo genético em um labirinto 10x10

A execução do algoritmo genético tem como parâmetros o tamanho do labirinto, tamanho da população inicial, taxa de crossover, taxa de mutação, número máximo de gerações, a cada quantas gerações um novo passo é dado no labirinto e a função de fitness utilizada para o cálculo ("EUCLIDEAN", "MANHATTAN").

In [ ]:
def calculate_and_generate(population, maze, start, finish, crossover_rate, mutation_rate, growth, fitness_function):
  population_size = len(population)

  population_fitness = []
  for pop in population:
    if (fitness_function == "EUCLIDEAN"):
      tmp = euclidean_fitness(pop, maze, start, finish)
    else:
      tmp = manhattan_fitness(pop, maze, start, finish)
    if(tmp == -1):
      return (True, pop)
    population_fitness.append((tmp, pop))

  population_fitness.sort(reverse=True)

  new_population = []
  crossover_tmp = []
  for _, pop in population_fitness:
    if(random.random() <= crossover_rate):
      crossover_tmp.append(pop)

      if(len(crossover_tmp) == 2):
        new_population.extend(crossover(crossover_tmp[0], crossover_tmp[1]))
        crossover_tmp = []

    if(random.random() <= mutation_rate):
      new_population.append(mutation(pop))

    if(len(new_population) >= population_size):
      new_population = new_population[:population_size]
      break

  i = 0
  while(len(new_population) < population_size):
    new_population.append(population_fitness[i][1])

    i += 1

  if (growth and len(new_population[0]) < len(maze)**2):
    for individual in new_population:
      individual.append(random.randint(0,3))

  mean_list = []
  for i in range(len(population_fitness)):
    mean_list.append(population_fitness[i][0])

  mean = np.mean(np.array(mean_list))
  return (False, new_population, mean)

def run(maze_size, population_size, crossover_rate, mutation_rate, max_generation, n_generation_growth, fitness_function = "EUCLIDEAN"):
  maze = maze_maker(maze_size)
  start, finish = start_finish_cells(maze_size)

  make_maze_viable(maze, start, finish)
  
  population = init_population(maze_size, population_size)
  generation = 1
  fitness_values = []
  while(generation <= max_generation):
    growth = (generation % n_generation_growth) == 0
    tmp = calculate_and_generate(population, maze, start, finish, crossover_rate, mutation_rate, growth, fitness_function)

    if(tmp[0]):
      return tmp[1], generation-1, fitness_values, maze, start, finish

    population = tmp[1]
    fitness_values.append(tmp[2])
    generation += 1

  return population[0], max_generation, fitness_values, maze, start, finish

In [ ]:
random.seed(SEED)

In [ ]:
chromosome, generation, fitness_values, maze, start, finish = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5)
for i in maze:
  print(i)
print("Start: ", start)
print("Finish: ", finish)
print("Chromosome: ", chromosome)
print("Generation: ", generation)
print("Fitness values: ", fitness_values)

# Gráfico com evolução do algoritmo genético (gerações x fitness)

In [ ]:
def show_graph(last_generation, fitness_values):    
  generations = [i+1 for i in range(last_generation)]
  plt.plot(generations, fitness_values)
  plt.grid()
  plt.ylabel("Fitness")
  plt.xlabel("Gerações")
  plt.title("Evolução do AG")
  plt.show()

In [ ]:
show_graph(generation, fitness_values)

No gráfico visualizado, vemos que a medida que as gerações passavam, a média de fitness da população tendeu a aumentar até a solução do labirinto. 

No gráfico não é representado a população/geração em que a solução foi encontrada pois o fitness do candidato que solucionou tenderia ao infinito.

# Figura do labirinto e melhor solução encontrada

In [ ]:
# Maze intermediary list representation.
def maze_cell_list(maze, start):
  maze_size = len(maze)
  maze_as_text_list = [ [' ' for col in range(maze_size+2)] for row in range(maze_size+2) ]
  #outlines || duplicated 'for' to preserve '-' over '|'
  for row in range(maze_size+2):
    maze_as_text_list[row][0] = '|'
    maze_as_text_list[row][maze_size+1] = '|'
  for col in range(maze_size+2):
    maze_as_text_list[0][col] = '-'
    maze_as_text_list[maze_size+1][col] = '-'  
  return maze_as_text_list

def print_maze(maze, maze_as_text_list, coordinates_, start, finish, animation=False):
  maze_as_text_list[start+1][1] = rep['start']
  maze_as_text_list[finish+1][len(maze)] = rep['finish']
  coordinates = coordinates_[:]
  coordinates.reverse()
  while(coordinates):
    coord = coordinates.pop()
    if (maze_as_text_list[coord[0]+1][coord[1]+1] != rep['start'] and
        maze_as_text_list[coord[0]+1][coord[1]+1] != rep['finish']): 
        maze_as_text_list[coord[0]+1][coord[1]+1] = rep['step']
    if (animation):
      clear_output(wait=True)
      print(maze_to_text(maze, maze_as_text_list))
      sleep(1)
  if (not animation): print(maze_to_text(maze, maze_as_text_list))

def maze_to_text(maze, maze_cell_list):
  maze_size = len(maze)
  maze_as_text = ""

  for row in range(1, maze_size+1):
    #top
    for col in range(1, maze_size+1):
      if (col == 1): maze_as_text += rep['cross']
      if (maze[row-1][col-1][2]): maze_as_text += ' '
      else: maze_as_text += rep['hwall']
      maze_as_text += rep['cross']
    maze_as_text += '\n'
    
    #right
    for col in range(1, maze_size+1):

      if (col==1): maze_as_text += rep['vwall']
      maze_as_text += maze_cell_list[row][col]
      
      if (maze[row-1][col-1][1]): maze_as_text += ' '

      else: maze_as_text += rep['vwall']
    
    maze_as_text += '\n'
  maze_as_text += rep['hwall']*(maze_size*2 +1 )
  return maze_as_text

#### Figura do labirinto com todos os passos feitos

In [ ]:
print("Chromosome: ", chromosome)
_, coordinates = get_coordinates_and_distance(chromosome, maze, start)
inter_rep_list = maze_cell_list(maze, start)
print_maze(maze, inter_rep_list, coordinates, start, finish )

#### Figura do labirinto com animação da execução dos passos do individuo.

In [ ]:
animated_inter_rep_list = maze_cell_list(maze, start)
print_maze(maze, animated_inter_rep_list, coordinates, start, finish, True)

# Variação dos parâmetros do algoritmo genético

Para efeitos de comparação das variações de parâmetros do algoritmo genético, reiniciaremos a seed utilzada para gerarmos os mesmos labirintos para os diferentes parâmetros.

Parâmetros base

---
Taxa de crossover == **0.8**

Taxa de mutação == **0.1**

Tamanho da população == **100**

Geração máxima para parada == **500**

Esses parâmetros foram obtidos através do artigo e foram utilizados nas seções anteriores.

Para termos uma média de gerações de convergência do AG, rodamos o algoritmo randômicamente por 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  _, generation, _, _, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Parâmetros variados

----

Taxa de crossover == ~0.8~ => **0.6**

Taxa de mutação == ~0.1~ => **0.3**

Tamanho da população == ~100~ => **80**

Geração máxima para parada == ~500~ => **1500**

Seguindo a lógica acima, rodamos o algoritmo com os parâmetros variados mais 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=80, crossover_rate=0.6, mutation_rate= 0.3, max_generation=1500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Aparentemente, a média de gerações para a convergência do Algoritmo Genético com os parâmetros variados tendeu a diminuir.

O gráfico a seguir é referente à ultima execução do algoritmo.



In [ ]:
show_graph(generation, fitness_values)

#### Utilizando outra função de fitness (Distância de Manhattan)

No exemplo a seguir utilizaremos a função fitness com Distância de Manhattan ao invés da Distância Euclideana.

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5, fitness_function= "MANHATTAN")
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Ao alterar a função fitness para utilizar a Distância de Manhattan, notamos uma piora na convergência do algoritmo genético com os mesmos parâmetros base (artigo).

O gráfico a seguir é referente à ultima execução do algoritmo.

In [ ]:
show_graph(generation, fitness_values)


#### Efeito da variação dos parâmetros na velocidade de convergência da solução


O algoritmo genético com a variação dos parâmetros e função fitness de Distância Euclidiana convergiu mais rapidamente que o algoritmo com os parâmetros base.

Enquanto o AG com os parâmetros base (artigo) e Distância Euclidiana como função fitness encontrou a solução em uma média de **50.74** gerações, o AG com parâmetros variados sugeridos por todos os integrantes do grupo convergiu em uma média de **45.50** gerações.

Em uma variação da função fitness para utilizar a Distância de Manhattan, o AG convergiu em uma média de **65.5** gerações.

